In [128]:
def export_to_csv(data_frame, file_name):
    """
    Export a DataFrame to a CSV file.

    Parameters:
        - data_frame: pandas DataFrame
        - file_name: str, name of the CSV file (without the extension)
    """
    #print date stamp - datetime.datetime.now()
    
    #t_stamp = str(datetime.datetime.now()).split(".")[0].replace(" ","_")
    
    file_path = f"{file_name}.csv"
    
    data_frame.to_csv(file_path, index=False)
    print(f"DataFrame successfully exported to {file_path}")

In [129]:
import os
import time
import pandas as pd

# pip install selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains

from bs4 import BeautifulSoup # pip install beautifulsoup4

import random


Accedemos a la pagina web de Mercadona 

In [130]:
df = pd.DataFrame(columns=['categoria', 'subcategoria', 'descripcion', "unidades", "precio", "code", "compare"])
df

df = pd.DataFrame(columns=['categoria', 'subcategoria', 'descripcion', "unidades", "precio", "code", "compare"])

In [131]:
supermecados = ["mercadona", "dia", "consum"]

#https://tienda.consum.es/es#!Home

In [132]:
supermercado = supermecados[0]

In [166]:
#ENTRAR EN MERCADONA:

# Start the driver with the options
driver = webdriver.Chrome()

actions = ActionChains(driver)

# Navigate to the login page
link = f'https://www.{supermercado}.es/'
driver.get(link)

#Reject/Accept cookies:
driver.find_element(By.XPATH, "//button[contains(text(),'Rechazar')]").click()

# Enter the postal code and submit
code="08015"

#Find the box elemet for postal code and input the code:
driver.find_element(By.CSS_SELECTOR, 'input[aria-label="Código postal"]').send_keys(code)

# click the botom code 
driver.find_element(By.CSS_SELECTOR, 'input.postal-code-form__button').click()

#Click on Categories Button(Wait til its visible)
WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.LINK_TEXT, "Categorías"))).click()

time.sleep(2)

#Save all categories into a variable:
category_links = driver.find_elements(By.CSS_SELECTOR, "span[class='category-menu__header']")

if category_links != 0:
    os.system("say categorias ok, entrando")
else:
    os.system("say categorias vacias")


for c in category_links:
    
    
    category = c.text
    #print(category)
    
    non = ["Aceite, especias y salsas", "Agua y refrescos", "Aperitivos", "Arroz, legumbres y pasta",
                    "Azúcar, caramelos y chocolate", "Bebé", "Bodega", "Cacao, café e infusiones", "Carne",
                    "Cereales y galletas", "Congelados", "Charcutería y quesos", "Cuidado del cabello", "Cuidado facial y Corporal"]
    
    if category in non:
        continue
    
    #Enter the category:
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, f"//label[text()='{category}']"))).click()
    
    #time.sleep(random.randint(10,30))
    
    #Target subcategories:
    subcategory_links = driver.find_elements(By.CSS_SELECTOR, ".category-item__link")
    
    if subcategory_links != 0:
        os.system("say subcategorias ok, entrando")
    else:
        os.system("say subcategorias vacias")
        
    #time.sleep()
        
    for s in subcategory_links:
        subcategory = s.text
        
        avoid = ["Aves y jamón cocido", "Bacón y salchichas", "Chopped y mortadela", "Embutido curado",
                "Jamón serrano", "Paté y sobrasada", "Queso curado, semicurado y tierno"]
        
        if subcategory in avoid:
            continue
        
        
        print(subcategory)
        
        
        time.sleep(random.randint(2,5))
        
        #Enter a subcategorie:
        WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, f"//button[text()='{subcategory}']"))).click()
        #print("subclicada")
        
        #time.sleep(1)
        
        # Wait for the products to load and saves them
        WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".product-cell")))

        #Extrae todos los productos(cajita que vimos )
        product_cells = driver.find_elements(By.CSS_SELECTOR, "div[data-test='product-cell']")
        
        os.system("say productos ok, entrando")

        for p in product_cells:
            product = p
            #print(product)
            
            #time.sleep(random.randint(2,8))
            
            des = product.find_element(By.CSS_SELECTOR, "h4[data-test='product-cell-name']").text
            unit = product.find_element(By.CSS_SELECTOR, "div[class='product-format product-format__size--cell']").text
            pr = product.find_element(By.CSS_SELECTOR, "div[class='product-price']").text
            
            '--------ANADIMOS URL----------------------------------'
            
            time.sleep(3) #este yo lo quitaria
            
            # Click on the "product-cell"
            
            WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "h4[data-test='product-cell-name']")))
            
            actions.move_to_element(product).perform()
            
            product.click()

            # Get the current URL of the page
            current_url = driver.current_url
            code = current_url.split("/")[4]
            
            time.sleep(2)
            
            
            WebDriverWait(driver, 30).until(EC.visibility_of_element_located((By.CSS_SELECTOR, "div[class='product-format product-format__size']")))
     
            info = driver.find_element(By.CSS_SELECTOR, "div[class='product-format product-format__size']").text
            compare = info.split("|")[-1]
            
            # Escape:
            while "categories" not in driver.current_url:
                driver.back()
                driver.find_element(By.CSS_SELECTOR, "body").send_keys(Keys.ESCAPE)
        
            nueva_fila = {
                'categoria':category , 
                'subcategoria': subcategory, 
                'descripcion': des,
                "unidades": unit,
                "precio":pr,
                "code": code,
                "compare" : compare
                }
            
            # Wait the second half of the time passed before clicking the next product
            #time.sleep(2)
            
            #time.sleep(random.randint(2,10))
  

            df = pd.concat([df, pd.DataFrame([nueva_fila])], ignore_index=True)
            #print("row inserted")
        
    time.sleep(200)

WebDriverException: Message: unknown error: net::ERR_NAME_NOT_RESOLVED
  (Session info: chrome=122.0.6261.112)
Stacktrace:
0   chromedriver                        0x000000010a3202a8 chromedriver + 4625064
1   chromedriver                        0x000000010a317e53 chromedriver + 4591187
2   chromedriver                        0x0000000109f16e6a chromedriver + 392810
3   chromedriver                        0x0000000109f0eba0 chromedriver + 359328
4   chromedriver                        0x0000000109efe7d1 chromedriver + 292817
5   chromedriver                        0x0000000109effc05 chromedriver + 297989
6   chromedriver                        0x0000000109efead2 chromedriver + 293586
7   chromedriver                        0x0000000109efdbe8 chromedriver + 289768
8   chromedriver                        0x0000000109efda6b chromedriver + 289387
9   chromedriver                        0x0000000109efc81c chromedriver + 284700
10  chromedriver                        0x0000000109efcbaa chrome

In [165]:
export_to_csv(df,"mercadona-05")

DataFrame successfully exported to mercadona-04.csv


In [164]:
df

,categoria,subcategoria,descripcion,unidades,precio,code,compare
0,Bodega,Cerveza sin alcohol,Cerveza sin alcohol Falke,6 botellines x 250 ml,"2,10 € /pack",86552,"1,40 €/L"
1,Bodega,Cerveza sin alcohol,Cerveza sin alcohol Falke,Botellín 250 ml,"0,35 € /ud.",86553,"1,40 €/L"
2,Bodega,Cerveza sin alcohol,"Cerveza 0,0% sin alcohol Falke",6 botellines x 250 ml,"2,10 € /pack",66394,"1,40 €/L"
3,Bodega,Cerveza sin alcohol,"Cerveza 0,0% sin alcohol Falke",Botellín 250 ml,"0,35 € /ud.",15990,"1,40 €/L"
4,Bodega,Cerveza sin alcohol,"Cerveza 0,0% sin alcohol tostada Mahou",6 botellines x 330 ml,"6,90 € /pack",66558,"3,485 €/L"
...,...,...,...,...,...,...,...
1793,Cuidado del cabello,Champú,Champú y acondicionador anticaspa Classic 2 en...,Bote 360 ml,"2,40 € /ud.",35399,"0,667 €/100 ml"
1794,Cuidado del cabello,Champú,Champú infantil Kids 3 en 1 Deliplus,Bote 300 ml,"1,70 € /ud.",44320,"0,567 €/100 ml"
1795,Cuidado del cabello,Champú,Loción infantil elimina liendres y piojos Deli...,Bote 150 ml,"4,50 € /ud.",67802,"3,00 €/100 ml"
1796,Cuidado del cabello,Champú,Champú Protect Triple Action Deliplus preventi...,Bote 300 ml,"2,50 € /ud.",67800,"0,834 €/100 ml"
